In [1]:
from helpers.models import Models
from helpers.llm_client import LLMClient
from helpers.parsers import license_parser
from helpers.functions import *
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score
# pd.set_option('display.max_colwidth', None)  # Show full column width

/home/jimbo/Desktop/GSoC24/repo/GSoC24/gsoc24env/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/jimbo/Desktop/GSoC24/repo/GSoC24/gsoc24env/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
create_license_dataset('extras/license_information/details')
client = LLMClient()
model = SentenceTransformer("all-mpnet-base-v2")
df = pd.read_csv('extras/dataset-labeled-1-75.csv')
lines_tuple = get_top_similar_license_lines(\
                code_text = df.loc[5, 'file_comments'],
                licenses_file_path = 'extras/license_information/license_dataset.csv',
                model = model,
                top_k = 10,
                embedding_approach='license-embedding',
                double_semantic_search = True
    )
lines_tuple

License dataset file created successfully at extras/license_information/license_dataset.csv


/home/jimbo/Desktop/GSoC24/repo/GSoC24/gsoc24env/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[('SPDX-License-Identifier: GPL-2.0',
  'GNU General Public License v2.0 only',
  'GPL-2.0'),
 ('Only ia64 requires this', 'Reciprocal Public License 1.5', 'RPL-1.5'),
 ('hugepage-mmap:** Example of using huge page memory in a user application using the mmap* system call.  Before running this application, make sure that the* administrator has mounted the hugetlbfs filesystem (on some directory* like /mnt) using the command mount -t hugetlbfs nodev /mnt. In this* example, the app is requesting memory of size 256MB that is backed by* huge pages.** For the ia64 architecture, the Linux kernel reserves Region number 4 for* huge pages.  That means that if one requires a fixed address, a huge page* aligned address starting with 0x800000... will be required.  If a fixed* address is not required, the kernel will select an address in the proper* range.* Other architectures, such as ppc64, i386 or x86_64 are not so constrained.',
  'Creative Commons Attribution Share Alike 4.0 International',
  '

## multi-qa-mpnet-base-dot-v1

In [ ]:
create_license_dataset('extras/license_information/details')
client = LLMClient()
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")
df = pd.read_csv('extras/dataset-labeled-1-75.csv')
df = extract_comments(df)
for index, row in df.loc[:75].iterrows():
    lines_tuple = get_top_similar_license_lines(\
                    code_text = row['file_comments'],
                    licenses_file_path = 'extras/license_information/license_dataset.csv',
                    model = model,
                    top_k = 10,
                    embedding_approach='license-embedding'
            )
    lines = [line[2] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'license_relevant_lines'] = lines
    lines = [line[3] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_names'] = lines
    lines = [line[4] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_ids'] = lines
for index, row in df.iterrows():
    df.loc[index, 'license_found'] = license_line_found(row['license_relevant_lines'], row['response'])
    df.loc[index, 'coverage'] = asses_coverage(row['license_relevant_lines'], row['response'])
print('accuracy:', accuracy_score([1] * 75, df.loc[0:74]['license_found']))
print('coverage: ', np.average(df.loc[0:74]['coverage']))

In [ ]:
miss_classifcations = df.loc[0:74][df['license_found'] == 0]
miss_classifcations[['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']]

## all-mpnet-base-v2

In [2]:
create_license_dataset('extras/license_information/details')
client = LLMClient()
model = SentenceTransformer("all-mpnet-base-v2")
df = pd.read_csv('extras/dataset-labeled-1-75.csv')
for index, row in df.loc[:75].iterrows():
    lines_tuple = get_top_similar_license_lines(\
                    code_text = row['file_comments'],
                    licenses_file_path = 'extras/license_information/license_dataset.csv',
                    model = model,
                    top_k = 10,
                    embedding_approach='license-embedding',
                    double_semantic_search = True
            )
    lines = [line[2] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'license_relevant_lines'] = lines
    lines = [line[3] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_names'] = lines
    lines = [line[4] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_ids'] = lines
for index, row in df[0:75].iterrows():
    df.loc[index, 'license_found'] = license_line_found(row['license_relevant_lines'], row['response'])
    df.loc[index, 'license_coverage'] = asses_coverage(row['license_relevant_lines'], row['response'])
    df.loc[index, 'predicted_license_accurate'] = predicted_license_found(row['predicted_license_ids'], row['scan results'])
    df.loc[index, 'predicted_licenses_covered'] = predicted_license_covered(row['predicted_license_ids'], row['scan results'])
print('License Line Found Accuracy:', accuracy_score([1] * 75, df.loc[0:74]['license_found']))
print('License Lines Found Coverage:', np.average(df.loc[0:74]['license_coverage']))
print('Predicted License Accuracy:', accuracy_score([1] * 75, df.loc[0:74]['predicted_license_accurate']))
print('Predicted Licenses Covered:', np.average(df.loc[0:74]['predicted_licenses_covered']))

License dataset file created successfully at extras/license_information/license_dataset.csv


/home/jimbo/Desktop/GSoC24/repo/GSoC24/gsoc24env/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


License Line Found Accuracy: 0.9866666666666667
License Lines Found Coverage: 92.6984126984127
Predicted License Accuracy: 0.6933333333333334
Predicted Licenses Covered: 63.777777777777786


In [5]:
pd.set_option('display.max_colwidth', None)  # Show full column width
df[df['predicted_license_accurate'] == 0][['scan results', 'predicted_license_ids']]

,scan results,predicted_license_ids
10,GPL,CAL-1.0-Combined-Work-Exception\nMIT-advertising\nDotseqn\nAAL\nPolyForm-Noncommercial-1.0.0\nHPND-export-US-acknowledgement\nCC-BY-NC-SA-3.0\nMulanPSL-1.0\nmailprio\nTAPR-OHL-1.0
48,ISC,0BSD\nCC-BY-NC-SA-2.0-UK\nMPL-2.0-no-copyleft-exception\nLZMA-SDK-9.22\nCECILL-C\nCECILL-2.1\nCECILL-C\nTAPR-OHL-1.0\nTPL-1.0\nlibselinux-1.0
49,ISC,0BSD\nGFDL-1.1-or-later\nMPEG-SSG\nSSPL-1.0
50,ISC,0BSD\ncopyleft-next-0.3.0\nHPND-doc-sell
51,ISC,0BSD\nCC-BY-NC-SA-3.0\nBrian-Gladman-2-Clause\nBitstream-Vera\nxlock
53,ISC,0BSD\nPolyForm-Noncommercial-1.0.0
54,ISC,0BSD\nIPL-1.0\nMcPhee-slideshow\nElastic-2.0
55,ISC,0BSD\nw3m\nPolyForm-Noncommercial-1.0.0
56,ISC,0BSD\ncheck-cvs\nCC-BY-NC-SA-2.0-UK\nHPND-Intel\nFTL\nTAPR-OHL-1.0
57,ISC,0BSD\nMPEG-SSG\nCECILL-C\nCommunity-Spec-1.0\nOGL-UK-2.0


In [3]:
print(df.loc[63, 'file_comments'])

sync fence wait tests*  Copyright 2015-2016 Collabora Ltd.**  Based on the implementation from the Android Open Source Project,**  Copyright 2012 Google, Inc**  Permission is hereby granted, free of charge, to any person obtaining a*  copy of this software and associated documentation files (the "Software"),*  to deal in the Software without restriction, including without limitation*  the rights to use, copy, modify, merge, publish, distribute, sublicense,*  and/or sell copies of the Software, and to permit persons to whom the*  Software is furnished to do so, subject to the following conditions:**  The above copyright notice and this permission notice shall be included in*  all copies or substantial portions of the Software.**  THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR*  IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,*  FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL*  THE AUTHORS OR COPYRIGHT HOLDERS BE

In [8]:
miss_classifcations = df.loc[0:74][df['coverage'] != 100]
miss_classifcations[['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']]

/tmp/ipykernel_6722/3738649951.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  miss_classifcations = df.loc[0:74][df['coverage'] != 100]


,file path,license_relevant_lines,response,predicted_license_names,scan results,predicted_license_ids
26,linux-master/tools/usb/usbip/INSTALL,"2006, 2007 Free Software Foundation, Inc.\nThi...",['This file is free documentation; the Free So...,FSF Unlimited License (with License Retention)...,FSFUL,FSFULLR\nFSFULLRWD\nFSFULLRWD\nElastic-2.0\nFS...
27,linux-master/tools/usb/ffs-aio-example/multibu...,This is free and unencumbered software release...,['This is free and unencumbered software relea...,The Unlicense\nPolyForm Noncommercial License ...,Unlicense MIT-style,Unlicense\nPolyForm-Noncommercial-1.0.0\nlsof\...
29,linux-master/tools/usb/ffs-aio-example/simple/...,This is free and unencumbered software release...,['This is free and unencumbered software relea...,The Unlicense\nAffero General Public License v...,MIT-style Unlicense,Unlicense\nAGPL-1.0-or-later\nLPPL-1.1\nIPL-1.0
31,linux-master/tools/usb/ffs-aio-example/multibu...,This is free and unencumbered software release...,['This is free and unencumbered software relea...,The Unlicense\nAffero General Public License v...,MIT-style Unlicense,Unlicense\nAGPL-1.0-or-later\nLPPL-1.1\nIPL-1.0
41,linux-master/tools/testing/selftests/vDSO/pars...,And here's the code.\nSymbol table\nparse_vdso...,['parse_vdso.c: Linux reference vDSO parser* W...,THOR Public License 1.0\nCommon Lisp LOOP Lice...,CC0-1.0,TPL-1.0\nLOOP\nRPSL-1.0\npnmstitch\nFDK-AAC\nM...
66,pytorch-main/torch/utils/hipify/hipify_python.py,"Permission is hereby granted, free of charge,...","['SPDX-License-Identifier: GPL-2.0-only', ' Pe...",X11 License\nX11 License\nUniversal Permissive...,MIT,X11\nX11\nUPL-1.0\nUPL-1.0\ncheck-cvs\nLPPL-1....
67,linux-master/tools/testing/selftests/sync/sw_s...,sw_sync abstraction** Copyright 2015-2016 Col...,['Copyright 2015-2016 Collabora Ltd.** Based ...,Universal Permissive License v1.0\nUnited K...,MIT,UPL-1.0\nOPL-UK-3.0
69,linux-master/tools/testing/selftests/sync/sync...,sync fence tests with one timeline* Copyright...,['Copyright 2015-2016 Collabora Ltd.** Based ...,Universal Permissive License v1.0\nCreative Co...,MIT,UPL-1.0\nCC-BY-NC-SA-2.0-UK\nMPL-2.0-no-copyle...
70,pytorch-main/torch/_appdirs.py,This is the MIT license\nThis file is directly...,"['SPDX-License-Identifier: GPL-2.0-only', 'Thi...",RealNetworks Public Source License v1.0\nUnive...,MIT,RPSL-1.0\nUPL-1.0\nTMate\nSSPL-1.0\nGFDL-1.1-o...
71,linux-master/tools/testing/selftests/sync/sync...,sync test runner* Copyright 2015-2016 Collabo...,['Copyright 2015-2016 Collabora Ltd.** Based ...,Universal Permissive License v1.0\nServer Side...,MIT,UPL-1.0\nSSPL-1.0


In [36]:
df.loc[0:74][df['license_found'] == 1][['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']]

/tmp/ipykernel_6722/1861196098.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.loc[0:74][df['license_found'] == 1][['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']]


,file path,license_relevant_lines,response,predicted_license_names,scan results,predicted_license_ids
0,linux-master/tools/testing/selftests/powerpc/b...,"Copyright 2015, Anton Blanchard, IBM Corp.\nSP...",['SPDX-License-Identifier: GPL-2.0-only'],Haskell Language Report License\nGNU General P...,GPL-2.0-only,HaskellReport\nGPL-2.0-only
1,linux-master/tools/testing/selftests/kvm/lib/x...,SPDX-License-Identifier: GPL-2.0-only\nSetup o...,['SPDX-License-Identifier: GPL-2.0-only'],GNU General Public License v2.0 only\nCeCILL-C...,GPL-2.0-only,GPL-2.0-only\nCECILL-C\nTPL-1.0\nBSD-Systemics...
2,linux-master/tools/testing/selftests/tc-testin...,!/bin/sh SPDX-License-Identifier: GPL-2.0,['SPDX-License-Identifier: GPL-2.0'],GNU General Public License v2.0 only,GPL-2.0-only,GPL-2.0
3,linux-master/tools/testing/selftests/pid_names...,# SPDX-License-Identifier: GPL-2.0\nCFLAGS += ...,['SPDX-License-Identifier: GPL-2.0'],GNU General Public License v2.0 only\nCeCILL F...,GPL-2.0-only,GPL-2.0\nCECILL-1.1\nSSPL-1.0\nCECILL-C\nCNRI-...
4,linux-master/tools/testing/selftests/hid/progs...,Copyright (c) 2022 Red hat\nSPDX-License-Ident...,['SPDX-License-Identifier: GPL-2.0'],pnmstitch License\nGNU General Public License ...,GPL-2.0-only,pnmstitch\nGPL-2.0\nSAX-PD\nODbL-1.0\nODbL-1.0...
...,...,...,...,...,...,...
70,pytorch-main/torch/_appdirs.py,This is the MIT license\nThis file is directly...,"['SPDX-License-Identifier: GPL-2.0-only', 'Thi...",RealNetworks Public Source License v1.0\nUnive...,MIT,RPSL-1.0\nUPL-1.0\nTMate\nSSPL-1.0\nGFDL-1.1-o...
71,linux-master/tools/testing/selftests/sync/sync...,sync test runner* Copyright 2015-2016 Collabo...,['Copyright 2015-2016 Collabora Ltd.** Based ...,Universal Permissive License v1.0\nServer Side...,MIT,UPL-1.0\nSSPL-1.0
72,linux-master/tools/testing/selftests/sync/sync.c,sync / sw_sync abstraction* Copyright 2015-20...,['Copyright 2015-2016 Collabora Ltd.** Based ...,Universal Permissive License v1.0\nRealNetwork...,MIT,UPL-1.0\nRPSL-1.0\n3D-Slicer-1.0
73,linux-master/tools/testing/selftests/sched/cs_...,SPDX-License-Identifier: GPL-2.0-only\nUse the...,"['SPDX-License-Identifier: GPL-2.0-only', 'Thi...",GNU General Public License v2.0 only\nGNU Less...,LGPL-2.1-only,GPL-2.0-only\nLGPL-2.1-or-later\nNAIST-2003\nl...


In [7]:
licenses = pd.read_csv('extras/license_information/license_dataset.csv')

## paraphrase-MiniLM-L3-v2

In [ ]:
create_license_dataset('extras/license_information/details')
client = LLMClient()
model = SentenceTransformer("paraphrase-MiniLM-L3-v2")
df = pd.read_csv('extras/dataset-labeled-1-75.csv')
for index, row in df.loc[:75].iterrows():
    lines_tuple = get_top_similar_license_lines(\
                    code_text = row['file_comments'],
                    licenses_file_path = 'extras/license_information/license_dataset.csv',
                    model = model,
                    top_k = 10,
                    embedding_approach='license-embedding'
            )
    lines = [line[2] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'license_relevant_lines'] = lines
    lines = [line[3] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_names'] = lines
    lines = [line[4] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_ids'] = lines
for index, row in df.iterrows():
    df.loc[index, 'license_found'] = license_line_found(row['license_relevant_lines'], row['response'])
    df.loc[index, 'coverage'] = asses_coverage(row['license_relevant_lines'], row['response'])
print('accuracy:', accuracy_score([1] * 75, df.loc[0:74]['license_found']))
print('coverage: ', np.average(df.loc[0:74]['coverage']))

In [ ]:
miss_classifcations = df.loc[0:74][df['license_found'] == 0]
miss_classifcations[['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']]

## all-MiniLM-L12-v2 

In [ ]:
create_license_dataset('extras/license_information/details')
client = LLMClient()
model = SentenceTransformer("all-MiniLM-L12-v2")
df = pd.read_csv('extras/dataset-labeled-1-75.csv')
for index, row in df.loc[:75].iterrows():
    lines_tuple = get_top_similar_license_lines(\
                    code_text = row['file_comments'],
                    licenses_file_path = 'extras/license_information/license_dataset.csv',
                    model = model,
                    top_k = 10,
                    embedding_approach='license-embedding'
            )
    lines = [line[2] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'license_relevant_lines'] = lines
    lines = [line[3] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_names'] = lines
    lines = [line[4] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_ids'] = lines
for index, row in df.iterrows():
    df.loc[index, 'license_found'] = license_line_found(row['license_relevant_lines'], row['response'])
    df.loc[index, 'coverage'] = asses_coverage(row['license_relevant_lines'], row['response'])
print('accuracy:', accuracy_score([1] * 75, df.loc[0:74]['license_found']))
print('coverage: ', np.average(df.loc[0:74]['coverage']))

In [ ]:
miss_classifcations = df.loc[0:74][df['license_found'] == 0]
miss_classifcations[['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']]

In [ ]:
idx = 1
print(df[df['license_found'] == 1][['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']].loc[idx])
print('----')
print(df.loc[idx, 'predicted_license_ids'])